In [1]:
%pip install tensorflow

  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 3.7/390.3 MB 54.1 MB/s eta 0:00:08
   -- ------------------------------------- 19.9/390.3 MB 54.6 MB/s eta 0:00:07
   --- ------------------------------------ 31.5/390.3 MB 55.4 MB/s eta 0:00:07
   ---- ----------------------------------- 41.9/390.3 MB 54.4 MB/s eta 0:00:07
   ----- ---------------------------------- 54.8/390.3 MB 56.3 MB/s eta 0:00:06
   ------ --------------------------------- 67.1/390.3 MB 57.0 MB/s eta 0:00:06
   -------- ------------------------------- 81.3/390.3 MB 58.2 MB/s eta 0:00:06
   --------- ------------------------------ 94.4/390.3 MB 59.0 MB/s eta 0:00:06
   ---------- ---------------------------- 107.5/390.3 MB 59.7 MB/s eta 0:00:05
   ------------ -------------------------- 121.1/390.3 MB 60.4 MB/s eta 0:00:05
   ------------- ------------------------- 133.2/390.3 MB 60.3 MB/s

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import glob

# Step 1: 整合所有 CSV 文件
file_paths = glob.glob("data/train.csv")  # 修改為您的實際文件路徑模式
all_data = pd.DataFrame()

for file in file_paths:
    data = pd.read_csv(file)
    all_data = pd.concat([all_data, data], ignore_index=True)

print(f"數據集大小: {all_data.shape}")

# Step 2: 數據預處理
# 假設數據集包含 '日期' 和 '目標值' 列，您需要根據實際數據進行修改
all_data['日期'] = pd.to_datetime(all_data['日期'])  # 確保日期列格式正確

# 按日期排序
all_data = all_data.sort_values(by='日期')

# 填補缺失值
all_data = all_data.fillna(method='ffill').fillna(method='bfill')

# 特徵工程（這部分需要根據您的數據具體修改）
all_data['year'] = all_data['日期'].dt.year
all_data['month'] = all_data['日期'].dt.month
all_data['day'] = all_data['日期'].dt.day

# 歸一化目標值
scaler = MinMaxScaler(feature_range=(0, 1))
all_data['目標值'] = scaler.fit_transform(all_data[['目標值']])

# Step 3: 數據分割
# 篩選數據集
train_data = all_data[(all_data['year'] >= 2014) & (all_data['year'] <= 2020)]
val_data = all_data[(all_data['year'] == 2021) | (all_data['year'] == 2022)]
test_data = all_data[all_data['year'] == 2023]

# 創建時間序列數據
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length, :-1])
        y.append(data[i + sequence_length, -1])
    return np.array(X), np.array(y)

sequence_length = 30  # 使用過去 30 天的數據進行預測

# 訓練數據
train_values = train_data[['目標值']].values
X_train, y_train = create_sequences(train_values, sequence_length)

# 驗證數據
val_values = val_data[['目標值']].values
X_val, y_val = create_sequences(val_values, sequence_length)

# 測試數據
test_values = test_data[['目標值']].values
X_test, y_test = create_sequences(test_values, sequence_length)

# Step 4: 構建 LSTM 模型
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, activation='relu'),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Step 5: 模型訓練
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1
)

# Step 6: 模型評估
y_test_pred = model.predict(X_test)

# 將數據反轉回原始尺度
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
y_test_pred = scaler.inverse_transform(y_test_pred)

mse = mean_squared_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print(f"Test MSE: {mse:.2f}")
print(f"Test R2: {r2:.2f}")

# Step 7: 保存模型
import joblib
joblib.dump(model, 'trained_lstm_model.h5')

In [ ]:
# Step 8: 可視化結果
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(y_test, label='True Values')
plt.plot(y_test_pred, label='Predictions')
plt.legend()
plt.title('LSTM Predictions vs True Values')
plt.show()